# **지능형 IoT 응용**
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 12. 지연 최적화

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML, 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어
  - 15장

---

- 임베디드 시스템의 처리 지연 시간
  - 임베디드 시스템은 컴퓨팅 능력이 크지 않기 때문에 신경망이 필요로 하는 계산에 다른 플랫폼보다 더 많은 시간이 걸릴 수 있음
  - 임베디드 시스템은 보통 실시간으로 센서 데이터 스트림을 받아 처리하는 방식으로 동작하기 때문에 계산 속도에 병목 현상이 발생하면 많은 문제가 생길 수 있음
  - 예를 들어 카메라 시야에 들어온 새를 관찰하는 것처럼 찰나에 발생할 수 있는 사건을 관찰한다고 가정해보자. 처리 시간이 너무 길면 카메라에서 이미지 캡처가 느리게 되어 이러한 사건 발생을 놓칠 수 있음


- 지연 시간 단축의 이점
  - 호출어 감지 예제에서 1초 오디오 데이터를 처리하는 동시에 0.1초 단위로 창을 옮겨가며 평균을 구하여 처리하는 방식처럼 센서 데이터 스트림에 여러 창을 중복시켜 반복 평균하는 방식으로 예측의 성능이 높아지는 경우도 있음. 이러한 경우 지연 시간을 줄이면 전체 정확도가 향상됨
  - 모델의 실행 속도를 높이면 더 낮은 CPU 주파수로 장치를 작동시키거나, 추론 사이사이에 휴면(sleep) 상태를 취하면서 전체 에너지 사용량을 줄일 수도 있음


- 지연 시간은 최적화를 위한 중요한 요소
  - 모델 실행 시간을 단축하는 데 사용할 수 있는 여러 가지 기술을 살펴보자


## 12.1 정말 중요한 문제인지 확인하기

- 신경망 코드는 전체 시스템 지연 시간의 작은 부분일 수 있으므로 모델 실행 처리 속도를 높여도 전체 시스템의 성능에 큰 영향을 미치지 않을 수도 있음


- 모델 처리 시간이 전체 실행 시간에 얼마나 큰 비중을 차지하는지 확인하는 것이 필요함
  - 모델 추론 부분(tflite::MicroInterpreter::Invoke())을 주석 처리하여 전체 지연 시간에 어떤 차이가 있는지 관찰하는 방식이 가능함
  - 타이머 로그나 프로파일러를 사용하여 지연 시간을 측정하여 그 차이를 확인할 수 있음


- 추론 실행 여부에 따른 지연 시간의 차이가 크지 않다면 딥러닝 모델 부분을 최적화하여 얻는 것이 많지 않을 것이므로 애플리케이션의 다른 부분에 중점을 둬야 함

## 12.2 하드웨어 변경

- 신경망 코드의 속도를 높여야 하는 경우 가장 먼저 질문해야 할 것
  - 더 강력한 하드웨어 장치를 사용할 수 있는지 여부


- 사용할 하드웨어 플랫폼 결정은 대부분 개발 초기에 이루어지거나 외부 요인에 의해 결정되기 때문에 대부분의 임베디드 제품에서는 가능하지 않지만, 소프트웨어 관점에서 하드웨어를 결정하면 많은 설계가 쉬워지기 때문에 고려할 만한 가치가 있음


- 선택의 여지가 있다면 가장 중요하게 고려할 요소는 일반적으로 에너지, 속도, 비용임
  - 사용 중인 하드웨어를 바꾸면서 증가하는 속도와 에너지, 비용 절감을 비교
  - 고사양의 하드웨어를 사용하게 되면 속도는 증가하지만, 그에 따라 에너지 소모량도 증가할 수 있고 비용도 증가하게 될 것임


- 에너지 소모, 비용 증가를 최소화하면서 더 빠른 속도를 제공하는 새로운 하드웨어 플랫폼이 있다면 좋은 대안이 될 수 있음

## 12.3 모델 개선

- 충분히 정확하지만 계산 횟수가 적은 새 모델을 만들 수 있다면 다른 코드를 전혀 변경하지 않고도 추론 속도를 높일 수 있음
  - 일반적으로 정확도를 희생하여 속도를 높이는 것은 가능하므로 처음부터 정확도가 높은 모델로 시작할 수 있다면 속도를 높이기 위한 절충의 범위가 넓어짐
  - 훈련 데이터를 개선하고 늘리는 데 시간을 투자하면 지연 시간 최적화와 같은 관련 없어 보이는 작업에도 영향을 미치며 개발 프로세스 전체를 개선할 수 있음


- 모델 아키텍처를 다룰 때 어떠한 작업의 속도를 높이기보다는 가능하면 작업을 완전히 없애는 것이 더 좋은 최적화임


- 단 머신러닝 모델은 입력 데이터를 받아 숫자 결과를 반환하는 기능적 '블랙박스'이기 때문에 기존 코드에서 알고리즘을 전환하는 것보다 머신러닝 모델 자체를 교체하는 것이 훨씬 쉬움
  - 훈련 스크립트에서 한 모델을 다른 모델로 쉽게 교체할 수 있음


- 사용 중인 모델에서 개별 레이어를 제거하여 실험하면서 효과를 관찰할 수도 있음
  - 신경망은 성능이 급격히 저하되지 않으니 다양한 변화를 시도하여 정확도와 지연 시간에 미치는 영향을 관찰하는 것이 좋음



### 12.3.1 모델 지연 시간 추정

- 신경망 모델을 대부분 시간을 대규모 행렬 곱셈이나 매우 유사한 연산을 실행하는 데 소비함


- 모든 입력값이 각 출력값에 대해 다른 가중치로 조정되어야 하므로 입력값 수에 네트워크의 각 레이어에서 나오는 출력값 수를 곱하면 연산 규모를 파악할 수 있음
  - 보통 네트워크에서 추론을 한 번 실행할 때 필요한 부동소수점 연산(FLOP)의 수를 연산의 단위로 함 (FLOPs: FLoating point Operations, FLOPS: FLoating point Operations Per Second)
  - 일반적으로 곱셉-덧셈 연산(보통 기계어 수준에서 하나의 명령으로 처리)은 두 개의 FLOP으로 계산되며 8비트 이하의 양자화 계산을 수행하는 경우도 FLOP으로 간주함


- 신경망에 필요한 FLOP 수는 레이어별로 수동으로 계산할 수 있음

- 예를 들어, 완전 연결 레이어(Fully-connected/Dense layer)의 경우
  - FLOPs = 2 * MAC
  - MAC(Multiply-Accumulate) = output.shape * input.shape (출력 벡터 크기 x 입력 벡터 크기)
 
- 아래 링크 참고 (convolution layer, pooling layer 등)
  - https://www.c-sharpcorner.com/article/how-to-optimize-a-neural-network/ 
  - https://www.thinkautonomous.ai/blog/?p=deep-learning-optimization


- FLOP은 신경망을 실행하는 데 걸리는 시간에 대한 대략적인 지표로 사용할 수 있음
  - 모델의 다른 모든 요소가 동일하다면 계산 횟수가 적은 모델이 FLOP 차이에 비례하여 더 빨리 실행됨
  - 예를 들어 1억 FLOP 모델이 2억 FLOP 모델보다 2배 빠른 속도로 실행될 것이라고 대략적으로 예상할 수 있음
  - 정확히 따지면 지연 시간에 영향을 줄 수 있는 특정 계층에 대해 소프트웨어가 얼마나 잘 최적화됐는지와 같은 다른 요소도 고려해야 하지만 뉴럴 네트워크 아키텍처를 평가하기 위한 좋은 출발점이며 하드웨어 플랫폼 성능을 예측하는 데 도움이 됨
  - 100ms 내에 1백만 FLOP 모델을 실행할 수 있는 하드웨어라면 1천만 FLOP이 필요한 다른 모델을 실행하는 데는 약 1초가 걸릴 것을 예상할 수 있음

 


### 12.3.2 모델 속도 개선 방법

- 모델 아키텍처 설계는 여전히 연구가 활발히 진행되는 분야이므로 초보자를 위한 가이드를 제공하는 것은 쉽지 않음


- 가장 좋은 출발점은 효율성을 염두에 두고 설계된 기존 모델을 찾은 다음 변경을 반복적으로 적용하며 실험하는 것임
  - 많은 모델에는 변경 가능하며 필요한 계산량에 영향을 주는 파라미터가 존재함. 이런 파라미터를 변경해보면서 속도의 차이를 확인해보는 것이 한 가지 방법
  - 각 레이어에 필요한 FLOP을 보고 특히 느린 레이어를 제거하거나 더 빠른 대안(일반 컨볼루션을 깊이 컨볼루션으로 대체하는 등)으로 변경하는 방법도 있음
  - 가능하다면 FLOP을 통해 추정하는 대신 장치에서 실행할 때 각 레이어의 실제 지연 시간을 살펴보는 것도 가치가 있음. 이를 위해서는 프로파일링 기술이 필요할 수 있음

## 12.4 양자화

- 신경망을 실행하려면 한번의 추론에 대해 수십만 또는 수백만 회의 계산이 필요할 수 있음
- 이렇게 복잡한 계산을 수행하는 대부분의 일반적인 프로그램은 수치 정밀도에 매우 민감하며 정밀도가 떨어지면 오류가 발생하고 부정확한 결과를 낼 수 있음


- 하지만 딥러닝 모델은 이와 달리, 중간 계산 중에 수치 정밀도의 손실에 대처할 수 있으며 전체적으로 정확한 최종 결과를 생성할 수 있음
  - 이러한 특성은 크고 노이즈가 가득한 입력 데이터를 쓰는 훈련 과정의 부산물로, 모델은 중요하지 않은 변화에 영향을 덜 받으며 중요한 패턴에 초점을 맞추도록 훈련되기 때문임


- 이것이 의미하는 것은 32비트 부동소수점 자료형이 추론에 있어서는 필요 이상으로 정확할 수 있다는 것임
  - 훈련에서는 가중치 업데이트가 많이 필요하므로 정확도가 더 필요하지만 훈련에서도 16비트 자료형이 널리 사용됨
  - 대부분의 추론 애플리케이션은 가중치와 활성화 값을 저장하기 위해 8비트 자료형만 사용해도 부동소수점을 쓸 때와 큰 차이 없는 결과를 생성할 수 있음


- 많은 임베디드 플랫폼이 8비트 곱셈-누산(Multiply-Accumulate) 명령을 지원하기 때문에 8비트 자료형을 사용하는 것이 큰 이점이 될 수 있음


#### 양자화 (Quantization)
- 32비트 부동소수점 숫자인 모델의 파라미터(예: 모델 가중치)를 나타내는 숫자의 정밀도를 줄여서 처리하는 것
  - 모델 크기가 감소하고 계산 속도가 향상됨



- TensorFlow Lite에서 지원하는 양자화 종류
  - https://www.tensorflow.org/lite/performance/model_optimization 
  - Post-training quantization (훈련 후 양자화) https://www.tensorflow.org/lite/performance/post_training_quantization
  - Quantization-Aware training (양자화 인식 훈련) https://www.tensorflow.org/model_optimization/guide/quantization/training 
 

- 훈련 후 양자화
  - 훈련 후 정수 양자화 (https://www.tensorflow.org/lite/performance/post_training_integer_quant)
  - 훈련 후 동적 범위 양자화 (https://www.tensorflow.org/lite/performance/post_training_quant)
  - 훈련 후 float16 양자화 (https://www.tensorflow.org/lite/performance/post_training_float16_quant)


- 훈련 후 정수 양자화
  - 32비트 부동소수점 숫자(예: 가중치 및 활성화 출력)를 가장 가까운 8비트 고정 소수점 숫자로 변환
  - 모델이 작아지고 추론 속도가 증가하여 마이크로 컨트롤러와 같은 저전력 장치에 유용
  - 모델 크기 축소 최대 75% (4배 더 작게), 3배 이상의 속도 향상
  - 활성화 출력도 양자화 하기 위해서는 활성화 레이어 출력 범위를 관찰할 수 있어야 함. 이를 위해 representative dataset이 필요


- 훈련 후 동적 범위 양자화
  - 가중치를 8비트로 양자화, 활성화 레이어는 부동소수점으로 유지
  - 활성화는 추론 시에 동적으로 양자화
  - 모델 크기 축소 최대 75%, 2-3배 속도 향상


- 훈련 후 float16 양자화
  - 16비트 부동소수점 숫자로 변환
  - 모델 크기 축소 최대 50%, 지연 시간과 정확도에 미치는 영향을 최소화하는 대신 모델 크기를 크게 줄일 수 있음
